In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
%matplotlib inline
import matplotlib.pyplot as plt

import sys 
import inspect
import seaborn as sns
import os

import perturbseq as perturb

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80, color_map='viridis')
sc.logging.print_versions()

scanpy==1.5.1 anndata==0.7.1 umap==0.4.4 numpy==1.17.2 scipy==1.4.1 pandas==0.24.2 scikit-learn==0.22 statsmodels==0.11.0 python-igraph==0.7.1 louvain==0.6.1 leidenalg==0.8.1


In [2]:
PROTEIN='KRAS'
DATA='/ahg/regevdata/projects/bn10_cancer_variants/bn10_oana_2018-06-22/results/2018-06-22/results/2020-02-14'
SUBSAMPLED=DATA+'/'+PROTEIN+'/subsampled/A549.'+PROTEIN

counts=20000
pref=SUBSAMPLED+'.channel_1to32.counts_per_cell'+str(counts)+'.1000cells.analysis'

FIGS=DATA+'/2020-09-15_Figures/'
os.system('mkdir -p '+FIGS)
print(FIGS)

/ahg/regevdata/projects/bn10_cancer_variants/bn10_oana_2018-06-22/results/2018-06-22/results/2020-02-14/2020-09-15_Figures/


In [3]:
adata=sc.read(SUBSAMPLED+'.channel_1to32.counts_per_cell'+str(counts)+'.regress_out_vbc.analysis.1000cells.new_format.h5ad',cache=False)
adata

Only considering the two last: ['.new_format', '.h5ad'].
Only considering the two last: ['.new_format', '.h5ad'].


AnnData object with n_obs × n_vars = 90262 × 1145 
    obs: 'batch', 'known', 'n_counts', 'n_genes', 'percent_mito', 'n_counts_downsampled', 'n_counts_original', 'vbc.counts', 'batch_num4', 'batch_num7', 'batch_num11', 'batch_num8', 'batch_num13', 'batch_num23', 'batch_num9', 'batch_num2', 'batch_num24', 'batch_num10', 'batch_num12', 'batch_num16', 'batch_num20', 'batch_num18', 'batch_num25', 'batch_num31', 'batch_num28', 'batch_num5', 'batch_num14', 'batch_num15', 'batch_num29', 'batch_num3', 'batch_num6', 'batch_num22', 'batch_num26', 'batch_num17', 'batch_num27', 'batch_num1', 'batch_num19', 'batch_num21', 'batch_num0', 'batch_num30', 'louvain', 'G1.S', 'G2.M', 'M', 'M.G1', 'S', 'phase.multi', 'guide', 'guide.compact', 'A146P', 'K117R', 'Y166H', 'K169K', 'K178K', 'T50P', 'A146V', 'L52F', 'K176Q', 'A155G', 'G60D', 'T158T', 'K88K', 'D119G', 'L159S', 'G13V', 'Q61P', 'D33E', 'S136N', 'K179R', 'T74T', 'G60S', 'Q99E', 'G13R', 'G12D', 'R164Q', 'I36M', 'C185Y', 'I163S', 'P34R', 'G60V', 'G12

In [4]:
#table
PROTEIN='KRAS'
OUT='/ahg/regevdata/projects/bn10_cancer_variants/bn10_oana_2018-06-22/results/2018-06-22/results/2020-02-14/'+PROTEIN+'/anno'
#original anno
anno=pd.read_csv(OUT+'/'+PROTEIN+'.anno.complete.csv',sep='\t')
anno.index=anno['Variant']
print(anno.head())

        Variant  Position From To mutation_type  Count_pancan  Count_ExAC  \
Variant                                                                     
G12D       G12D      12.0    C  T        Single        1782.0         NaN   
G12V       G12V      12.0    C  A        Single        1539.0         NaN   
G12C       G12C      12.0    C  A        Single        1110.0         NaN   
G13D       G13D      13.0    C  T        Single         582.0         NaN   
G12A       G12A      12.0    C  G        Single         353.0         NaN   

        control_status control_color Library_synthesis  long_name  
Variant                                                            
G12D           unknown         white                in  KRAS_G12D  
G12V           unknown         white                in  KRAS_G12V  
G12C           unknown         white                in  KRAS_G12C  
G13D           unknown         white    not in library        NaN  
G12A           unknown         white                

In [5]:
anno['control_status2']=list(anno['control_status'])
for i in range(anno.shape[0]):
    var=anno.index[i]
    
    if var in ['unassigned','WT']:
        continue
        
    pos=str(int(anno['Position'][i]))
    
    
    if pos in ['12','13','61']:
        anno['control_status2'][i]='hotspot'
anno.loc['unassigned','control_status2']='unknown'

/ahg/regevdata/users/oursu/software/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [7]:
anno.loc['unassigned','control_color']='slateblue'

anno['text_color']='black'

anno.loc[anno['control_status']=='ExAC','text_color']='gray'
anno.loc[anno['control_status']=='synonymous','text_color']='gray'
anno.loc[anno['control_status2']=='hotspot','text_color']='red'

anno.loc[anno['control_status']=='ExAC','control_color']='lightgray'
anno.loc[anno['control_status']=='synonymous','control_color']='gray'


In [7]:
anno.head()

,Variant,Position,From,To,mutation_type,Count_pancan,Count_ExAC,control_status,control_color,Library_synthesis,long_name,control_status2
Variant,,,,,,,,,,,,
G12D,G12D,12.0,C,T,Single,1782.0,NaN,unknown,white,in,KRAS_G12D,hotspot
G12V,G12V,12.0,C,A,Single,1539.0,NaN,unknown,white,in,KRAS_G12V,hotspot
G12C,G12C,12.0,C,A,Single,1110.0,NaN,unknown,white,in,KRAS_G12C,hotspot
G13D,G13D,13.0,C,T,Single,582.0,NaN,unknown,white,not in library,NaN,hotspot
G12A,G12A,12.0,C,G,Single,353.0,NaN,unknown,white,in,KRAS_G12A,hotspot


In [8]:
anno['control_status2'].value_counts()

unknown       55
hotspot       21
synonymous    16
ExAC          10
Name: control_status2, dtype: int64

In [8]:
anno.to_csv(OUT+'/'+PROTEIN+'.anno.complete2.csv',sep='\t')